In [14]:
import param
import panel as pn
import numpy as np
import pandas as pd
import holoviews as hv
import time
import bokeh
import xarray
import hvplot
import hvplot.xarray
import mth5
from mth5.mth5 import MTH5
import mt_metadata

pn.extension(sizing_mode = 'stretch_width')

In [15]:
displayed_columns = ['survey', 'station', 'run',
                     #'latitude', 'longitude', 'elevation',
                     'component',
                     'start', 'end', 'n_samples', 'sample_rate',
                     'measurement_type',
                     #'azimuth', 'tilt',
                     #'units'
                     ]

hv.opts(hv.opts.Rectangles(alpha = 0.5))


class Tsvi(pn.template.FastListTemplate):
    
    
    plot_button           = pn.widgets.Button(name = 'Plot')
    

    
    cpu_usage             = pn.indicators.Number(
                                                 name="CPU",
                                                 value=0,
                                                 format="{value}%",
                                                 colors=[(50, "green"), (75, "orange"), (100, "red")],
                                                 font_size="13pt",
                                                 title_size="8pt",
                                                 width=50,
                                             )
    
    memory_usage          = pn.indicators.Number(
                                                 name="Memory",
                                                 value=0,
                                                 format="{value}%",
                                                 colors=[(50, "green"), (75, "orange"), (100, "red")],
                                                 font_size="13pt",
                                                 title_size="8pt",
                                                 width=50,
                                             )
    
    streaming_resources = param.Boolean(default=False)
        
        
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.select_button         = pn.widgets.Button(name = 'Select Files')
        self.cache = {}
        self.files = pn.widgets.FileSelector(name = 'Files',
                                             directory = '~',
                                             file_pattern = '*.h5'
                                             )
        self.file_paths = {}
        self.channels = pn.widgets.MultiSelect(objects = [],
                                               name = 'Channels',
                                               placeholder = 'Please choose a file')
        self.channel_preview = pn.widgets.Select(options = self.channels.value,
                                                 name = 'Channel info',
                                                 placeholder = 'Please select a channel')
        self.channels.link(self.channel_preview, value = 'options')
        self.channel_summary = pd.DataFrame(columns = displayed_columns)
        self.summary_display = pn.widgets.DataFrame(self.channel_summary, height = 500)
        
        self.plots = []
        self.graphs = pn.Column()
        
        self.tab1 = pn.Column(self.files,
                              self.select_button,
                              name = 'Folders')
        self.tab2 = pn.Column(
                    pn.Row(pn.Column(self.channels,
                                     self.plot_button),
                           pn.Column(self.channel_preview),
                           ),
                    self.summary_display,
                    name = 'Channels')
        self.tab3 = pn.Column(self.graphs,
                              name = 'Plot')
        self.tabs = pn.Tabs(self.tab1,
                            self.tab2,
                            self.tab3,
                            closable = True,
                            dynamic = True)
        self.datashade = pn.widgets.Checkbox(name = 'Datashade',
                                             default = True)
        self.shared_axes = pn.widgets.Checkbox(name = 'Shared Axes',
                                               default = True)
        #TODO: Holoviews currently doesn't have individual sharing of x and y axes.
        #      It may be possible to do this, but it is not as trivial as we first thought.
        #      https://discourse.holoviz.org/t/how-to-share-only-the-x-axis-between-a-curve-and-a-quadmesh/3335
        #      https://discourse.holoviz.org/t/how-to-only-link-share-only-the-x-axis/93
        #      https://github.com/holoviz/holoviews/issues/49
        #self.shared_y_axis = pn.widgets.Checkbox()
        self.annotator = hv.annotate.instance()
        self.note_boxes = self.annotator(hv.Rectangles(data = None).opts(alpha = 0.5))
        
        
        
        
        
        
        self.main.append(self.tabs)

        
        self.sidebar.append(self.cpu_usage)
        self.sidebar.append(self.memory_usage)
        self.sidebar.append(self.datashade)
        self.sidebar.append(self.shared_axes)
        #self.start_resource_stream()
        
        self.select_button.on_click(self.update_channels)
        self.plot_button.on_click(self.make_and_display_plots)
        self.channels.link(self.summary_display, callbacks = {'value': self.display_channel_summary})
        
        return
        
    
    
  #  def start_resource_stream(self):
  #      if self.streaming_resources:
  #          retur
  #      def resouce_usage_psutil():
  #          return psutil.virtual_memory().percent, psutil.cpu_percent()
  #      def stream_resources():
  #          mem, cpu = resouce_usage_psutil()
  #          self.cpu_usage.value = cpu
  #          self.memory_usage.value = mem
  #      pn.state.add_periodic_callback(stream_resources, period=1000, count=None)
  #      self.streaming_resources = True
     
    
    
    
    def update_channels(self, *args, **kwargs):
        new_channels = []
        for file_path in self.files.value:
            file_name = file_path.split('/')[-1] #This might not work on Windows
            self.file_paths[file_name] = file_path
            m = MTH5()
            m.open_mth5(file_path, mode = 'r')
            df = m.channel_summary.to_dataframe()
            m.close_mth5()
            df['file'] = file_name
            df['channel_path'] = (df['file'] + '/' + df['station'] + '/' + df['run'] + '/' + df['component'])
            df.set_index('channel_path', inplace = True)
            self.cache[file_name] = df
            new_channels.extend(self.cache[file_name].index)
        self.channels.options = list(new_channels)
        return
        
    def display_channel_summary(self, target,  event):
        display_df = pd.DataFrame()
        for channel in event.new:
            display_df = pd.concat([display_df,(tsvi.cache[channel.split('/')[0]].loc[[channel], displayed_columns])])
        target.value = display_df
        return
    
    

    def make_plots(self):
        used_files = []
        new_cards  = []
        for selected_channel in self.channels.value:
            file_name = selected_channel.split('/')[0]
            if file_name not in used_files:
                used_files.append(file_name)
        for file in used_files:
            m = MTH5()
            m.open_mth5(self.file_paths[file], mode = 'r')
            for selected_channel in self.channels.value:
                selected_file, station, run, channel = selected_channel.split('/')
                if selected_file == file:
                    data = m.get_channel(station, run, channel).to_channel_ts().to_xarray()
                    plot = pn.bind(hvplot.hvPlot(data),
                                   datashade = self.datashade,
                                   shared_axes = self.shared_axes)
                    
                    plot2 = hv.Curve(data)

                    plot_tab = pn.Pane(plot, name = run + '/' + channel)
                    
                    
                    note_tab = pn.Row(self.annotator.compose(plot2, self.note_boxes))
                    new_card = pn.Card(pn.Tabs(plot_tab
                                               , note_tab
                                               ),
                                       title = selected_channel)
                    
                    new_cards.append(new_card)
            m.close_mth5()
        self.plots = new_cards
        return
                                      
    def display_plots(self):
        for plot in self.plots:
            self.graphs.append(plot)
        return
        
    def make_and_display_plots(self, *args, **kwargs):
        self.make_plots()
        self.display_plots()
        return





In [16]:
tsvi = Tsvi()
tsvi.show()

2022-09-02T22:07:17 [line 2529] param.MultiSelect07031.__db_print - WARNING: Setting non-parameter attribute objects=[] using a mechanism intended only for parameters


2022-09-02T22:07:17 [line 2529] param.Channels.__db_print - WARNING: Setting non-parameter attribute placeholder=Please choose a file using a mechanism intended only for parameters


2022-09-02T22:07:17 [line 2529] param.Channel info.__db_print - WARNING: Setting non-parameter attribute placeholder=Please select a channel using a mechanism intended only for parameters


2022-09-02T22:07:17 [line 2529] param.Datashade.__db_print - WARNING: Setting non-parameter attribute default=True using a mechanism intended only for parameters


2022-09-02T22:07:17 [line 2529] param.Shared Axes.__db_print - WARNING: Setting non-parameter attribute default=True using a mechanism intended only for parameters
Launching server at http://localhost:62867


2022-09-02 22:07:45,328 [line 726] mth5.mth5.MTH5.close_mth5 - INFO: Flushing and closing /Users/ianpatterson/test12rr.h5
/opt/anaconda3/envs/tsvi/lib/python3.9/site-packages/mth5-0.2.5-py3.9.egg/mth5/timeseries/channel_ts.py:83: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  dt_index = pd.date_range(
2022-09-02 22:07:56,449 [line 726] mth5.mth5.MTH5.close_mth5 - INFO: Flushing and closing /Users/ianpatterson/test12rr.h5


In [21]:
"""

Some things to be aware of between Datashader and Annotators. Datashader converts a holoviews.element
into a holoviews.core.spaces.DynamicMap. When .compose() -ing an Annotator, it requires elements, not DynamicMaps.
This means that the annotator won't work on a Datashaded plot.

One work around is to create a second plot that does not dynamically change between element and DynamicMap
with the push of a button and have this plot be used for the annotator. This works but it means that there are
two different plot objects created for each plot and doubles the loading time for each plot.
"""

"\n\nSome things to be aware of between Datashader and Annotators. Datashader converts a holoviews.element\ninto a holoviews.core.spaces.DynamicMap. When .compose() -ing an Annotator, it requires elements, not DynamicMaps.\nThis means that the annotator won't work on a Datashaded plot.\n\nOne work around is to create a second plot that does not dynamically change between element and DynamicMap\nwith the push of a button and have this plot be used for the annotator. This works but it means that there are\ntwo different plot objects created for each plot and doubles the loading time for each plot.\n"

In [19]:
tsvi.plots[0]

Card(sizing_mode='stretch_width', title='test12rr.h5/test1/001/ex'...)
    [0] Tabs(active=1, sizing_mode='stretch_width')
        [0] ParamFunction(function, name='001/ex', sizing_mode='stretch_width')
        [1] Row(sizing_mode='stretch_width')
            [0] HoloViews(Layout, sizing_mode='stretch_width')

In [20]:
tsvi.plots[0]

Card(sizing_mode='stretch_width', title='test12rr.h5/test1/001/ex'...)
    [0] Tabs(active=1, sizing_mode='stretch_width')
        [0] ParamFunction(function, name='001/ex', sizing_mode='stretch_width')
        [1] Row(sizing_mode='stretch_width')
            [0] HoloViews(Layout, sizing_mode='stretch_width')